In [12]:
import os
from PIL import Image

def get_image_dimensions(image_folder):
    dimensions = set()
    max_width = 0
    max_height = 0

    for filename in os.listdir(image_folder):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(image_folder, filename)
            try:
                with Image.open(image_path) as img:
                    w, h = img.size
                    dimensions.add((w, h))
                    if w > max_width:
                        max_width = w
                    if h > max_height:
                        max_height = h
            except:
                print(f"Erreur avec {image_path}")

    return dimensions, (max_width, max_height)

folders = [
    'data/train/fake',
    'data/train/real',
    'data/test/fake',
    'data/test/real'
]

for folder in folders:
    dims, max_dim = get_image_dimensions(folder)
    print(f"--- {folder} ---")
    print(f"Unique image dimensions: {dims}")
    print(f"Max dimensions: {max_dim}\n")


--- data/train/fake ---
Unique image dimensions: {(266, 268), (217, 263), (258, 264), (269, 264), (266, 277), (250, 260), (258, 273), (269, 273), (247, 273), (235, 280), (278, 269), (251, 252), (259, 265), (251, 261), (239, 259), (270, 274), (237, 326), (450, 470), (228, 268), (262, 270), (251, 270), (365, 493), (240, 242), (263, 253), (612, 370), (255, 249), (255, 258), (242, 297), (612, 406), (254, 271), (224, 270), (231, 269), (246, 267), (265, 271), (254, 280), (231, 278), (266, 254), (266, 263), (235, 257), (269, 259), (266, 272), (258, 268), (269, 268), (235, 266), (408, 612), (239, 254), (228, 254), (262, 265), (228, 272), (240, 237), (261, 269), (261, 278), (240, 255), (612, 374), (243, 260), (254, 257), (273, 270), (254, 275), (212, 269), (223, 269), (246, 271), (258, 263), (247, 263), (247, 272), (258, 272), (269, 272), (280, 269), (235, 279), (250, 277), (257, 276), (208, 271), (272, 274), (328, 327), (251, 251), (239, 258), (261, 264), (261, 273), (284, 275), (242, 269), (2

In [21]:
import os
from PIL import Image

folder = "data/train/fake"
total = 0
small_images = 0

for filename in os.listdir(folder):
    if filename.lower().endswith((".png", ".jpg", ".jpeg")):
        total += 1
        path = os.path.join(folder, filename)
        with Image.open(path) as img:
            w, h = img.size
            if w <= 256 and h <= 256:
                small_images += 1

ratio = small_images / total
print(f"Ratio d'images ≤256×256 : {ratio:.2%} ({small_images}/{total})")


Ratio d'images ≤256×256 : 100.00% (32019/32019)


In [14]:
import os
from PIL import Image
import shutil

# Dossier source
src_dir = "data/train/fake"
# Dossier où déplacer les images trop grandes (optionnel)
dest_dir = "data/train/fake_large"

os.makedirs(dest_dir, exist_ok=True)

max_size = 256
total = 0
moved = 0

for filename in os.listdir(src_dir):
    filepath = os.path.join(src_dir, filename)
    
    try:
        with Image.open(filepath) as img:
            width, height = img.size
            total += 1
            if width > max_size or height > max_size:
                # Déplacer au lieu de supprimer
                shutil.move(filepath, os.path.join(dest_dir, filename))
                moved += 1
    except Exception as e:
        print(f"Erreur sur {filename}: {e}")

print(f"Total images : {total}")
print(f"Images déplacées (>256 px) : {moved}")
print(f"Images restantes : {total - moved}")


Total images : 41594
Images déplacées (>256 px) : 1571
Images restantes : 40023


In [15]:
import os
import shutil
from PIL import Image
import random

# Chemins
train_dir = "data/train"
val_dir = "data/val"

# Pourcentage pour validation
val_ratio = 0.2

# Créer dossiers val si n'existent pas
for cls in ["fake", "real"]:
    os.makedirs(os.path.join(val_dir, cls), exist_ok=True)

# Parcours des classes
for cls in ["fake", "real"]:
    cls_train_path = os.path.join(train_dir, cls)
    cls_val_path = os.path.join(val_dir, cls)

    # Lister les fichiers
    images = [f for f in os.listdir(cls_train_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # Filtrer les images ≤ 256×256
    small_images = []
    for img_name in images:
        img_path = os.path.join(cls_train_path, img_name)
        with Image.open(img_path) as img:
            if img.width <= 256 and img.height <= 256:
                small_images.append(img_name)

    # Mélanger pour randomisation
    random.shuffle(small_images)

    # Nombre pour validation
    n_val = int(len(small_images) * val_ratio)
    val_images = small_images[:n_val]

    # Déplacer les images dans val
    for img_name in val_images:
        src = os.path.join(cls_train_path, img_name)
        dst = os.path.join(cls_val_path, img_name)
        shutil.move(src, dst)

    print(f"{cls}: {len(small_images)-n_val} train, {n_val} val")


fake: 32019 train, 8004 val
real: 39052 train, 9763 val


---

#### now we have 4 folders with images of size ≤256×256 , we should first resize them to a common size (256×256) and then we can start training our model.

In [2]:
import os
from PIL import Image

folders = {
    "train": "data/train",
    "val": "data/val",
    
}

target_size = (256, 256)

def resize_and_crop(image, size):
    # Resize en conservant le ratio
    img_ratio = image.width / image.height
    target_ratio = size[0] / size[1]

    if img_ratio > target_ratio:
        # trop large, resize par hauteur
        new_height = size[1]
        new_width = int(img_ratio * new_height)
    else:
        # trop haute, resize par largeur
        new_width = size[0]
        new_height = int(new_width / img_ratio)

    image = image.resize((new_width, new_height), Image.Resampling.LANCZOS)

    # Crop central
    left = (new_width - size[0]) // 2
    top = (new_height - size[1]) // 2
    right = left + size[0]
    bottom = top + size[1]

    return image.crop((left, top, right, bottom))

for split, path in folders.items():
    print(f"Processing {split}...")
    for label in os.listdir(path):
        label_path = os.path.join(path, label)
        if not os.path.isdir(label_path):
            continue
        for img_name in os.listdir(label_path):
            img_path = os.path.join(label_path, img_name)
            try:
                with Image.open(img_path) as img:
                    img = resize_and_crop(img, target_size)
                    img.save(img_path)
            except Exception as e:
                print(f"Erreur sur {img_path}: {e}")

print("✅ Toutes les images sont maintenant à 256x256 !")


Processing train...
Processing val...
✅ Toutes les images sont maintenant à 256x256 !
